In [1]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [2]:
df = pd.read_csv('dataset_augmented.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,...,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm,Unnamed: 78
0,0,0,0,0,2.798293,0.769563,-0.528015,1.951881,-0.431077,-0.641827,...,0,0,0,1,0,0,2.753333,599.0,612.000000,13.000000
1,1,1,1,1,0.055436,-0.361432,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.600000,530.0,613.000000,83.000000
2,2,2,2,2,0.782458,0.017999,-0.528015,0.600491,-0.431077,-0.641827,...,0,0,0,1,0,0,3.500000,595.0,610.000000,15.000000
3,3,4,4,4,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.520000,520.0,612.000000,92.000000
4,4,5,5,5,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,1,0,0,2.660000,590.0,612.000000,22.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,212,214,214,214,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,570.0,605.000000,35.000000
213,213,215,215,215,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,562.0,605.000000,43.000000
214,214,216,216,216,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,5.433333,549.0,589.666667,40.666667
215,215,217,217,217,-0.423738,-0.631411,-0.201566,-0.304661,-0.085945,-0.600008,...,0,0,0,0,0,0,2.000000,523.0,597.000000,74.000000


In [3]:
# Input for ML models

input_col = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_chloroindium oxalate',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium myristate',
             'x0_indium oxalate',
             'x0_indium palmitate',
             'x0_indium trifluoroacetate',
             'x0_indium tris(N,N-diisopropylacetamidinato)',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphine gas',
             'x1_phosphorus trichloride',
             'x1_sodium phosphide',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x1_white phosphorus',
             'x2_4-ethylpyridine',
             'x2_None',
             'x2_dimethylformamide',
             'x2_mesitylene',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x6_dodecanethiol',
             'x7_None',
             'x7_zinc acetate',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc octanoate',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_superhydride',
             'x8_oleic acid',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide'
            ]

output_col = ['diameter_nm', 'abs_nm', 'emission_nm']

X = df[input_col]

Y = df[output_col]

# Random Sampling


In [6]:
sampling = ("8/92", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True)
    ET_regr = ExtraTreesRegressor(n_estimators=2,
                            max_features=20,     
                            random_state=23).fit(X_train, Y_train)
            
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = mean_absolute_error(Y_test, pd.DataFrame(ET_Y_pred))
    print(sampling[i], ": ", ET_MAE)

8/92 :  15.939351850308645
10/90 :  15.985732321464647
15/85 :  14.608905725252521
20/80 :  17.742613636868686
25/75 :  18.763979797626263
30/70 :  17.01247474806397
40/60 :  21.64336526181354
50/50 :  24.80715596365953


In [36]:
sampling = ("8/92", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True)
    DT_regr = DecisionTreeRegressor(max_depth=13,
                                max_features=10,
                                random_state=10)
            
    DT_regr.fit(X_train, Y_train)            
    DT_Y_pred = DT_regr.predict(X_test)
    DT_MAE = mean_absolute_error(Y_test, pd.DataFrame(DT_Y_pred))
    print(sampling[i], ": ", DT_MAE)

8/92 :  24.919795523996914
10/90 :  17.865781791125357
15/85 :  20.18804066250324
20/80 :  21.600651726084806
25/75 :  22.00050504949495
30/70 :  21.547946129427604
40/60 :  21.20543668467434
50/50 :  24.917664070502827


In [ ]:
DT_regr = DecisionTreeRegressor(max_depth=13,
                                max_features=9,
                                random_state=10)
            
DT_regr.fit(X_train, Y_train)            
DT_Y_pred = DT_regr.predict(X_test)

# Stratified Sampling


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

df2 = df.sort_values(by = 'emission_nm')



In [8]:
df2


,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,...,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm,Unnamed: 78
180,180,182,182,182,-0.299814,0.426616,0.190172,-0.304661,-0.258511,0.612751,...,0,0,0,0,0,0,3.766667,405.0,470.0,65.0
181,181,183,183,183,-0.299814,0.426616,0.190172,-0.304661,0.086621,0.612751,...,0,0,0,0,0,0,3.666667,416.0,475.0,59.0
40,40,41,41,41,-0.175889,-0.448993,0.124882,0.071430,0.259187,-0.641827,...,0,0,0,0,0,0,2.000000,430.0,477.0,47.0
98,98,99,99,99,-0.175889,-0.448993,0.124882,0.071430,0.259187,-0.641827,...,0,0,0,0,0,0,2.000000,430.0,477.0,47.0
48,48,49,49,49,-0.588970,-0.667895,0.124882,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,1.766667,460.0,480.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,52,53,53,53,-0.588970,-0.521960,0.034782,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,4.633333,631.0,680.0,49.0
17,17,18,18,18,-0.341122,-0.594927,-0.103632,-0.304661,0.220839,-0.429943,...,0,0,0,0,0,0,4.533333,626.0,680.0,54.0
207,207,209,209,209,1.253370,1.448160,-0.528015,3.813777,-0.431077,-0.641827,...,0,0,0,1,0,0,4.200000,626.0,729.0,103.0
208,208,210,210,210,1.253370,1.448160,-0.528015,3.813777,-0.431077,-0.641827,...,0,0,0,1,0,0,4.800000,689.0,765.0,76.0


In [9]:
bins = [450, 500, 550, 600, 650, 700, 800]
df2['binned'] = pd.cut(df2['emission_nm'], bins)
df2

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,...,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm,Unnamed: 78,binned
180,180,182,182,182,-0.299814,0.426616,0.190172,-0.304661,-0.258511,0.612751,...,0,0,0,0,0,3.766667,405.0,470.0,65.0,"(450, 500]"
181,181,183,183,183,-0.299814,0.426616,0.190172,-0.304661,0.086621,0.612751,...,0,0,0,0,0,3.666667,416.0,475.0,59.0,"(450, 500]"
40,40,41,41,41,-0.175889,-0.448993,0.124882,0.071430,0.259187,-0.641827,...,0,0,0,0,0,2.000000,430.0,477.0,47.0,"(450, 500]"
98,98,99,99,99,-0.175889,-0.448993,0.124882,0.071430,0.259187,-0.641827,...,0,0,0,0,0,2.000000,430.0,477.0,47.0,"(450, 500]"
48,48,49,49,49,-0.588970,-0.667895,0.124882,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,1.766667,460.0,480.0,20.0,"(450, 500]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,52,53,53,53,-0.588970,-0.521960,0.034782,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,4.633333,631.0,680.0,49.0,"(650, 700]"
17,17,18,18,18,-0.341122,-0.594927,-0.103632,-0.304661,0.220839,-0.429943,...,0,0,0,0,0,4.533333,626.0,680.0,54.0,"(650, 700]"
207,207,209,209,209,1.253370,1.448160,-0.528015,3.813777,-0.431077,-0.641827,...,0,0,1,0,0,4.200000,626.0,729.0,103.0,"(700, 800]"
208,208,210,210,210,1.253370,1.448160,-0.528015,3.813777,-0.431077,-0.641827,...,0,0,1,0,0,4.800000,689.0,765.0,76.0,"(700, 800]"


In [10]:
sampling = ("5/92", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True, stratify=df2['binned'])
    ET_regr = ExtraTreesRegressor(n_estimators=5,
                            max_features=20,     
                            random_state=23).fit(X_train, Y_train)
            
    ET_regr.fit(X_train, Y_train)            
    ET_Y_pred = ET_regr.predict(X_test)
    ET_MAE = mean_absolute_error(Y_test, pd.DataFrame(ET_Y_pred))
    print(ET_MAE)

9.810484846666673
11.43398989505051
12.61587878666667
13.522388887272731
14.151652523030313
14.503747473670037
15.841951467867181
20.198988787461776


In [51]:
sampling = ("8/92", "10/90", "15/85", "20/80", "25/75", "30/70", "40/60", "50/50")
test_size = (0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5)
for i in range(0, 8):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size[i], random_state=45, shuffle=True, stratify=df2['binned'])
    DT_regr = DecisionTreeRegressor(max_depth=13,
                                max_features=10,
                                random_state=10)
            
    DT_regr.fit(X_train, Y_train)            
    DT_Y_pred = DT_regr.predict(X_test)
    DT_MAE = mean_absolute_error(Y_test, pd.DataFrame(DT_Y_pred))
    print(DT_MAE)

13.674188712768961
17.232171717037037
16.71080808178451
17.399915826430973
16.842418209456905
21.700267757299184
26.82226955194501
23.190723751804274
